In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("SPY_prices.csv", parse_dates=["Date"]).sort_values("Date").reset_index(drop=True)

# Signal: last 3 daily moves were positive (Close higher than previous Close)
df["up"] = df["Close"].diff() > 0
df["signal_buy"] = df["up"] & df["up"].shift(1) & df["up"].shift(2)

# Entry today close, exit next trading day close (shift(-1) = next row). [web:137]
df["entry_price"] = df["Close"]
df["exit_price"] = df["Close"].shift(-1)

# --- enforce "no overlapping trades" (since hold is exactly 1 day) ---
signal_idx = df.index[df["signal_buy"]].to_list()

taken = []
blocked_until = -1
for i in signal_idx:
    if i <= blocked_until:
        continue
    if pd.isna(df.loc[i, "exit_price"]):   # can't exit on last row
        continue
    taken.append(i)
    blocked_until = i + 1  # blocks the next day from starting a new trade

trades = df.loc[taken, ["Date", "entry_price", "exit_price"]].copy()
trades = trades.rename(columns={"Date": "EntryDate"})

# $1 per trade, not reinvested
trades["Contribution"] = 1.0
trades["Shares"] = trades["Contribution"] / trades["entry_price"]
trades["ValueAtExit"] = trades["Shares"] * trades["exit_price"]
trades["Profit"] = trades["ValueAtExit"] - trades["Contribution"]

# Exit date is next trading day
trades["ExitDate"] = trades["EntryDate"].shift(-1)  # just for labeling; optional
# Better: compute ExitDate from df directly:
trades["ExitDate"] = df.loc[taken, "Date"].values
trades["ExitDate"] = df.loc[[i+1 for i in taken], "Date"].values

trades.head()
trades.to_csv('3dayMomentumProfitablilty.CSV', index=False)


In [2]:
#markers to determine time-series where the trades are executed 
import pandas as pd
import numpy as np

df = pd.read_csv("SPY_prices.csv", parse_dates=["Date"]).sort_values("Date").reset_index(drop=True)

# --- signals ---
df["up"] = df["Close"].diff() > 0
df["signal_buy"] = df["up"] & df["up"].shift(1) & df["up"].shift(2)

# --- pick trades with no overlap (hold = 1 day) ---
signal_idx = df.index[df["signal_buy"]].to_list()

entry_idx = []
blocked_until = -1
for i in signal_idx:
    if i <= blocked_until:
        continue
    if i + 1 >= len(df):   # can't exit (no next day)
        continue
    entry_idx.append(i)
    blocked_until = i + 1  # blocks next day from being a new entry

exit_idx = [i + 1 for i in entry_idx]

# --- add markers to the *full daily* dataframe ---
df["EntryFlag"] = False
df["ExitFlag"] = False
df.loc[entry_idx, "EntryFlag"] = True   # boolean-style assignment via .loc [web:148]
df.loc[exit_idx, "ExitFlag"] = True     # boolean-style assignment via .loc [web:148]

# Optional: store trade-level info on those specific days
df["TradeID"] = np.nan
df.loc[entry_idx, "TradeID"] = range(1, len(entry_idx) + 1)
df.loc[exit_idx, "TradeID"] = range(1, len(exit_idx) + 1)

df["EntryPrice"] = np.nan
df["ExitPrice"] = np.nan
df.loc[entry_idx, "EntryPrice"] = df.loc[entry_idx, "Close"].values
df.loc[exit_idx, "ExitPrice"] = df.loc[exit_idx, "Close"].values

# If you want per-trade profit stamped on the exit day (since PnL realized at exit)
df["Profit"] = np.nan
profits = (1.0 / df.loc[entry_idx, "Close"].values) * df.loc[exit_idx, "Close"].values - 1.0
df.loc[exit_idx, "Profit"] = profits

# Save enriched daily dataset for plotting/inspection
df.to_csv("SPY_prices_with_momentum_marks.csv", index=False)

df.head(15)


,Date,Close,up,signal_buy,EntryFlag,ExitFlag,TradeID,EntryPrice,ExitPrice,Profit
0,1993-01-29,43.93750,False,False,False,False,NaN,NaN,NaN,NaN
1,1993-02-01,44.25000,True,False,False,False,NaN,NaN,NaN,NaN
2,1993-02-02,44.34375,True,False,False,False,NaN,NaN,NaN,NaN
3,1993-02-03,44.81250,True,True,True,False,1.0,44.8125,NaN,NaN
4,1993-02-04,45.00000,True,True,False,True,1.0,NaN,45.0,0.004184
5,1993-02-05,44.96875,False,False,False,False,NaN,NaN,NaN,NaN
6,1993-02-08,44.96875,False,False,False,False,NaN,NaN,NaN,NaN
7,1993-02-09,44.65625,False,False,False,False,NaN,NaN,NaN,NaN
8,1993-02-10,44.71875,True,False,False,False,NaN,NaN,NaN,NaN
9,1993-02-11,44.93750,True,False,False,False,NaN,NaN,NaN,NaN
